In [1]:
import pandas as pd
import numpy as np
import pickle 
from datetime import datetime
pd.set_option('display.max_columns', 300)
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import seaborn as sns

## Step 1: Read in hold out data, scalers, and best model

In [2]:
df = pd.read_csv('kc_house_data_test_features.csv', index_col=0)

In [3]:
df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [4]:
df.columns

Index(['id', 'date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')

## Step 2: Feature Engineering for holdout set

In [5]:
## Feature engineering
df['date'] = [x[:8] for x in df.date]
df.date = df.date.apply(lambda x: datetime.strptime(x, '%Y%m%d'))


In [6]:
current_year = datetime.now().year
df.yr_built = df.yr_built.astype('int64')
df['house_age'] = current_year - df['yr_built']

In [7]:

df['house_age_cat'] = df['house_age'].apply(lambda x: ('old' if x >= 50 else 'middle') if x >= 15 else 'new')

In [8]:
df.replace({'bedrooms': {33: 3}}, inplace=True)

In [9]:
df.replace({'bathrooms': {0: 0.25}}, inplace=True)

In [10]:
df['outdoor_space'] = df['sqft_lot'] - (df['sqft_above']/df['floors'])

In [11]:
df.replace({'sqft_lot': {1105: 1200}}, inplace=True)

In [12]:
df['multilevel'] = np.where(df['floors']> 1, 1, 0)

In [13]:
df['basement'] = np.where(df['sqft_basement']> 1, 1, 0)

In [14]:
df['renovated'] = np.where(df['yr_renovated']> 1, 1, 0)

In [15]:
df.drop('date', axis=1, inplace=True)

In [16]:
df.drop('sqft_basement', axis=1, inplace=True)

In [17]:
df.drop('yr_renovated', axis=1, inplace=True)

In [18]:
df.drop(columns=['floors', 'yr_built', 'lat', 'long', 'sqft_lot'], inplace=True)

In [19]:
df.drop(columns=['sqft_above'], inplace=True)

In [20]:
df = pd.get_dummies(df, columns=['condition', 'grade', 'zipcode'], drop_first=True)

In [21]:
df.shape

(4323, 97)

In [22]:
df.head()

,id,bedrooms,bathrooms,sqft_living,waterfront,view,sqft_living15,sqft_lot15,house_age,house_age_cat,outdoor_space,multilevel,basement,renovated,condition_2,condition_3,condition_4,condition_5,grade_4,grade_5,grade_6,grade_7,grade_8,grade_9,grade_10,grade_11,grade_12,grade_13,zipcode_98002,zipcode_98003,zipcode_98004,zipcode_98005,zipcode_98006,zipcode_98007,zipcode_98008,zipcode_98010,zipcode_98011,zipcode_98014,zipcode_98019,zipcode_98022,zipcode_98023,zipcode_98024,zipcode_98027,zipcode_98028,zipcode_98029,zipcode_98030,zipcode_98031,zipcode_98032,zipcode_98033,zipcode_98034,zipcode_98038,zipcode_98039,zipcode_98040,zipcode_98042,zipcode_98045,zipcode_98052,zipcode_98053,zipcode_98055,zipcode_98056,zipcode_98058,zipcode_98059,zipcode_98065,zipcode_98070,zipcode_98072,zipcode_98074,zipcode_98075,zipcode_98077,zipcode_98092,zipcode_98102,zipcode_98103,zipcode_98105,zipcode_98106,zipcode_98107,zipcode_98108,zipcode_98109,zipcode_98112,zipcode_98115,zipcode_98116,zipcode_98117,zipcode_98118,zipcode_98119,zipcode_98122,zipcode_98125,zipcode_98126,zipcode_98133,zipcode_98136,zipcode_98144,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
0,1974300020,4,2.50,2270,0,0,2020,10918,54,old,9960.0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1974300020,4,2.50,2270,0,0,2020,10918,54,old,9960.0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3630020380,3,2.50,1470,0,0,1470,1576,16,middle,1199.0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1771000290,3,1.75,1280,0,0,1160,10565,45,middle,15170.0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5126310470,4,2.75,2830,0,0,2830,7916,16,middle,6711.0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
df.drop(columns=['house_age_cat'], inplace=True)

In [24]:
contin_vars = ['bedrooms', 'bathrooms', 'sqft_living', 'waterfront', 'view', 'sqft_living15', 'sqft_lot15']

In [25]:
df['grade_3'] = ""

In [26]:
df['sqft_above'] = ""

In [27]:
df['grade_3'] = pd.to_numeric(df['grade_3'],errors='coerce')

In [28]:
df['sqft_above'] = pd.to_numeric(df['sqft_above'],errors='coerce')

In [29]:
df['grade_3'] = df['grade_3'].fillna(0)

In [30]:
df['sqft_above'] = df['sqft_above'].fillna(0)

In [31]:
df.shape

(4323, 98)

In [32]:

features = ['bedrooms', 'bathrooms', 'sqft_living', 'waterfront', 'view',
            'sqft_living15', 'sqft_lot15', 'house_age', 
        'outdoor_space', 'multilevel', 'basement', 'renovated',
       'condition_2', 'condition_3', 'condition_4', 'condition_5', 'grade_3',
       'grade_4', 'grade_5', 'grade_6', 'grade_7', 'grade_8', 'grade_9',
       'grade_10', 'grade_11', 'grade_12', 'grade_13', 'zipcode_98002',
       'zipcode_98003', 'zipcode_98004', 'zipcode_98005', 'zipcode_98006',
       'zipcode_98007', 'zipcode_98008', 'zipcode_98010', 'zipcode_98011',
       'zipcode_98014', 'zipcode_98019', 'zipcode_98022', 'zipcode_98023',
       'zipcode_98024', 'zipcode_98027', 'zipcode_98028', 'zipcode_98029',
       'zipcode_98030', 'zipcode_98031', 'zipcode_98032', 'zipcode_98033',
       'zipcode_98034', 'zipcode_98038', 'zipcode_98039', 'zipcode_98040',
       'zipcode_98042', 'zipcode_98045', 'zipcode_98052', 'zipcode_98053',
       'zipcode_98055', 'zipcode_98056', 'zipcode_98058', 'zipcode_98059',
       'zipcode_98065', 'zipcode_98070', 'zipcode_98072', 'zipcode_98074',
       'zipcode_98075', 'zipcode_98077', 'zipcode_98092', 'zipcode_98102',
       'zipcode_98103', 'zipcode_98105', 'zipcode_98106', 'zipcode_98107',
       'zipcode_98108', 'zipcode_98109', 'zipcode_98112', 'zipcode_98115',
       'zipcode_98116', 'zipcode_98117', 'zipcode_98118', 'zipcode_98119',
       'zipcode_98122', 'zipcode_98125', 'zipcode_98126', 'zipcode_98133',
       'zipcode_98136', 'zipcode_98144', 'zipcode_98146', 'zipcode_98148',
       'zipcode_98155', 'zipcode_98166', 'zipcode_98168', 'zipcode_98177',
       'zipcode_98178', 'zipcode_98188', 'zipcode_98198', 'zipcode_98199']

df_features = df[features]

In [33]:
poly_2 = PolynomialFeatures(degree=2, include_bias=False)
poly2_data = poly_2.fit_transform(df_features[contin_vars])
poly2_columns = poly_2.get_feature_names(df_features.columns)
df_poly2 = pd.DataFrame(poly2_data, columns=poly2_columns)

In [34]:
df_poly2.shape
df_poly2.columns

Index(['bedrooms', 'bathrooms', 'sqft_living', 'waterfront', 'view',
       'sqft_living15', 'sqft_lot15', 'bedrooms^2', 'bedrooms bathrooms',
       'bedrooms sqft_living', 'bedrooms waterfront', 'bedrooms view',
       'bedrooms sqft_living15', 'bedrooms sqft_lot15', 'bathrooms^2',
       'bathrooms sqft_living', 'bathrooms waterfront', 'bathrooms view',
       'bathrooms sqft_living15', 'bathrooms sqft_lot15', 'sqft_living^2',
       'sqft_living waterfront', 'sqft_living view',
       'sqft_living sqft_living15', 'sqft_living sqft_lot15', 'waterfront^2',
       'waterfront view', 'waterfront sqft_living15', 'waterfront sqft_lot15',
       'view^2', 'view sqft_living15', 'view sqft_lot15', 'sqft_living15^2',
       'sqft_living15 sqft_lot15', 'sqft_lot15^2'],
      dtype='object')

In [35]:
cat_vars = [x for x in df_features.columns if x not in contin_vars]

In [36]:
df_poly_cat = pd.concat([df_poly2, df_features[cat_vars]] , axis =1)

In [37]:
df_poly_cat.shape

(4323, 124)

In [38]:
df_poly_cat.head()

,bedrooms,bathrooms,sqft_living,waterfront,view,sqft_living15,sqft_lot15,bedrooms^2,bedrooms bathrooms,bedrooms sqft_living,bedrooms waterfront,bedrooms view,bedrooms sqft_living15,bedrooms sqft_lot15,bathrooms^2,bathrooms sqft_living,bathrooms waterfront,bathrooms view,bathrooms sqft_living15,bathrooms sqft_lot15,sqft_living^2,sqft_living waterfront,sqft_living view,sqft_living sqft_living15,sqft_living sqft_lot15,waterfront^2,waterfront view,waterfront sqft_living15,waterfront sqft_lot15,view^2,view sqft_living15,view sqft_lot15,sqft_living15^2,sqft_living15 sqft_lot15,sqft_lot15^2,house_age,outdoor_space,multilevel,basement,renovated,condition_2,condition_3,condition_4,condition_5,grade_3,grade_4,grade_5,grade_6,grade_7,grade_8,grade_9,grade_10,grade_11,grade_12,grade_13,zipcode_98002,zipcode_98003,zipcode_98004,zipcode_98005,zipcode_98006,zipcode_98007,zipcode_98008,zipcode_98010,zipcode_98011,zipcode_98014,zipcode_98019,zipcode_98022,zipcode_98023,zipcode_98024,zipcode_98027,zipcode_98028,zipcode_98029,zipcode_98030,zipcode_98031,zipcode_98032,zipcode_98033,zipcode_98034,zipcode_98038,zipcode_98039,zipcode_98040,zipcode_98042,zipcode_98045,zipcode_98052,zipcode_98053,zipcode_98055,zipcode_98056,zipcode_98058,zipcode_98059,zipcode_98065,zipcode_98070,zipcode_98072,zipcode_98074,zipcode_98075,zipcode_98077,zipcode_98092,zipcode_98102,zipcode_98103,zipcode_98105,zipcode_98106,zipcode_98107,zipcode_98108,zipcode_98109,zipcode_98112,zipcode_98115,zipcode_98116,zipcode_98117,zipcode_98118,zipcode_98119,zipcode_98122,zipcode_98125,zipcode_98126,zipcode_98133,zipcode_98136,zipcode_98144,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
0,4.0,2.50,2270.0,0.0,0.0,2020.0,10918.0,16.0,10.00,9080.0,0.0,0.0,8080.0,43672.0,6.2500,5675.0,0.0,0.0,5050.0,27295.00,5152900.0,0.0,0.0,4585400.0,24783860.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4080400.0,22054360.0,119202724.0,54,9960.0,0,1,0,0,1,0,0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,4.0,2.50,2270.0,0.0,0.0,2020.0,10918.0,16.0,10.00,9080.0,0.0,0.0,8080.0,43672.0,6.2500,5675.0,0.0,0.0,5050.0,27295.00,5152900.0,0.0,0.0,4585400.0,24783860.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4080400.0,22054360.0,119202724.0,54,9960.0,0,1,0,0,1,0,0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3.0,2.50,1470.0,0.0,0.0,1470.0,1576.0,9.0,7.50,4410.0,0.0,0.0,4410.0,4728.0,6.2500,3675.0,0.0,0.0,3675.0,3940.00,2160900.0,0.0,0.0,2160900.0,2316720.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2160900.0,2316720.0,2483776.0,16,1199.0,1,1,0,0,1,0,0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3.0,1.75,1280.0,0.0,0.0,1160.0,10565.0,9.0,5.25,3840.0,0.0,0.0,3480.0,31695.0,3.0625,2240.0,0.0,0.0,2030.0,18488.75,1638400.0,0.0,0.0,1484800.0,13523200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1345600.0,12255400.0,111619225.0,45,15170.0,0,1,0,0,1,0,0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4.0,2.75,2830.0,0.0,0.0,2830.0,7916.0,16.0,11.00,11320.0,0.0,0.0,11320.0,31664.0,7.5625,7782.5,0.0,0.0,7782.5,21769.00,8008900.0,0.0,0.0,8008900.0,22402280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8008900.0,22402280.0,62663056.0,16,6711.0,1,0,0,0,1,0,0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [39]:
df_poly_cat.shape

(4323, 124)

In [40]:
list(df_poly_cat.columns)

['bedrooms',
 'bathrooms',
 'sqft_living',
 'waterfront',
 'view',
 'sqft_living15',
 'sqft_lot15',
 'bedrooms^2',
 'bedrooms bathrooms',
 'bedrooms sqft_living',
 'bedrooms waterfront',
 'bedrooms view',
 'bedrooms sqft_living15',
 'bedrooms sqft_lot15',
 'bathrooms^2',
 'bathrooms sqft_living',
 'bathrooms waterfront',
 'bathrooms view',
 'bathrooms sqft_living15',
 'bathrooms sqft_lot15',
 'sqft_living^2',
 'sqft_living waterfront',
 'sqft_living view',
 'sqft_living sqft_living15',
 'sqft_living sqft_lot15',
 'waterfront^2',
 'waterfront view',
 'waterfront sqft_living15',
 'waterfront sqft_lot15',
 'view^2',
 'view sqft_living15',
 'view sqft_lot15',
 'sqft_living15^2',
 'sqft_living15 sqft_lot15',
 'sqft_lot15^2',
 'house_age',
 'outdoor_space',
 'multilevel',
 'basement',
 'renovated',
 'condition_2',
 'condition_3',
 'condition_4',
 'condition_5',
 'grade_3',
 'grade_4',
 'grade_5',
 'grade_6',
 'grade_7',
 'grade_8',
 'grade_9',
 'grade_10',
 'grade_11',
 'grade_12',
 'grade_1

## Step 3: Predict the holdout set

In [41]:
scaler = pickle.load(open('scaler.pickle', 'rb'))

In [42]:
final_scaler = StandardScaler()

scaled_data = final_scaler.fit_transform(df_poly_cat)

In [43]:
final_model = pickle.load(open('model.pickle', 'rb'))

In [44]:
with open('model.pickle', 'rb') as file:
    final_result = pickle.load(file)
final_result = final_result.predict(scaled_data)

In [45]:
final_result

array([539644.09782578, 539644.09782578, 365880.53642869, ...,
       316857.24758039, 336656.68051609, 316742.83352573])

## Step 4: Export your predictions

In [46]:
pd.DataFrame(final_result).to_csv('housing_preds_Christian_Corrales.csv')

In [47]:
result = pd.read_csv('/Users/B_A_S_i_C/Desktop/nyc-mhtn-ds-120720-lectures/Phase_2/Phase2_project/housing_preds_Christian_Corrales.csv', index_col=0)